# Conexão com o Oracle

## Instalando PIP

In [1]:
!pip install oracledb  # exclamação não se usa mais, agora eh % (pois ele é mais curto)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 29.5 MB/s eta 0:00:00


In [3]:
%pip install cx_Oracle  # exclamação não se usa mais, agora eh %

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 4.7 MB/s eta 0:00:00


## Instalando Bibliotecas

In [51]:
import oracledb
import cx_Oracle
import getpass # pegar password através de um input
import pandas as pd
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np

## Acessando o Banco

In [ ]:
# Entrando na conexão
pw = getpass.getpass("Enter password: ")

connection = oracledb.connect(
    user="RM553653",
    password=pw,
    dsn="oracle.fiap.com.br:1521/orcl")

print("Successfully connected to Oracle Database")

cursor = connection.cursor()

cursor.execute("SELECT username FROM all_users ORDER BY username")
for username in cursor:
    print(username)

## Iniciando as operações

* Criando uma Tabela

In [ ]:
# Criando uma nova Tabela
cursor.execute("""
    CREATE TABLE minha_nova_tabela (
        id INT PRIMARY KEY,
        nome VARCHAR2(50),
        descricao VARCHAR2(255)
    )
""")

* Explorando o Banco

In [ ]:
# Mostrando todas as tabelas
cursor.execute("SELECT table_name FROM user_tables")
for table in cursor:
    print(table)

In [ ]:
# Dando nome
results = cursor.execute("SELECT table_name FROM user_tables")
for abacate in results:
  print(abacate)

In [22]:
results

<oracledb.Cursor on <oracledb.Connection to RM553653@oracle.fiap.com.br:1521/orcl>>

In [23]:
results.description

[('TABLE_NAME', <DbType DB_TYPE_VARCHAR>, 128, 128, None, None, False)]

* Criação das tabelas

In [26]:
# Criação das tabelas
try:
    cursor.execute("""
        CREATE TABLE clientes (
            cliente_id INT PRIMARY KEY,
            nome VARCHAR2(100),
            email VARCHAR2(100)
        )
    """)

    cursor.execute("""
        CREATE TABLE produtos (
            produto_id INT PRIMARY KEY,
            nome VARCHAR2(100),
            preco DECIMAL(10,2)
        )
    """)

    cursor.execute("""
        CREATE TABLE vendedores (
            vendedor_id INT PRIMARY KEY,
            nome VARCHAR2(100)
        )
    """)

    cursor.execute("""
        CREATE TABLE vendas (
            venda_id INT PRIMARY KEY,
            cliente_id INT,
            produto_id INT,
            vendedor_id INT,
            quantidade INT,
            data_venda DATE,
            FOREIGN KEY (cliente_id) REFERENCES clientes(cliente_id),
            FOREIGN KEY (produto_id) REFERENCES produtos(produto_id),
            FOREIGN KEY (vendedor_id) REFERENCES vendedores(vendedor_id)
        )
    """)

    connection.commit()
    print("Tabelas criadas com sucesso.")
except cx_Oracle.DatabaseError as e:
    print("Erro ao criar tabelas:", e)
finally:
    cursor.close()
    connection.close()

Tabelas criadas com sucesso.


In [ ]:
cursor.execute("SELECT table_name FROM user_tables")
for table in cursor:
    print(table)

* Inserindo dados na Tabela

In [36]:
pw = getpass.getpass("Enter password: ")

connection = oracledb.connect(
    user="RM553653",
    password=pw,
    dsn="oracle.fiap.com.br:1521/orcl")

print("Successfully connected to Oracle Database")

cursor = connection.cursor()

Enter password: ··········
Successfully connected to Oracle Database


In [38]:
# Inserindo dados nas Tabelas

# Inserindo dados em 'produtos'
for i in range(1, 11):
    cursor.execute(f"""
        INSERT INTO produtos (produto_id, nome, preco)
        VALUES ({i}, 'Produto Eletrônico {i}', {random.uniform(100.0, 1000.0):.2f})
    """)

# Inserindo dados em 'clientes'
for i in range(1, 31):
    cursor.execute(f"""
        INSERT INTO clientes (cliente_id, nome, email)
        VALUES ({i}, 'Cliente Genérico {i}', 'cliente{i}@exemplo.com')
    """)

# Inserindo dados em 'vendedores'
for i in range(1, 11):
    cursor.execute(f"""
        INSERT INTO vendedores (vendedor_id, nome)
        VALUES ({i}, 'Vendedor {i}')
    """)

# Inserindo dados em 'vendas' com valores aleatórios
for i in range(1, 201):
    data_venda = datetime.now() - timedelta(days=random.randint(1, 365))
    cursor.execute(f"""
        INSERT INTO vendas (venda_id, cliente_id, produto_id, vendedor_id, quantidade, data_venda)
        VALUES ({i}, {random.randint(1, 30)}, {random.randint(1, 10)}, {random.randint(1, 10)}, {random.randint(1, 5)}, TO_DATE('{data_venda.strftime('%Y-%m-%d')}', 'YYYY-MM-DD'))
    """)

connection.commit()
print("Dados inseridos com sucesso.")

Dados inseridos com sucesso.


* Criando uma Viwer com Join

In [85]:
# Em DESC do Valor Total
try:
    cursor.execute("""
        SELECT v.venda_id, c.nome AS cliente_nome, p.nome AS produto_nome, p.preco, v.quantidade,
        (p.preco * v.quantidade) AS valor_total_venda, vd.nome AS vendedor_nome, v.data_venda
        FROM vendas v
        JOIN clientes c ON v.cliente_id = c.cliente_id
        JOIN produtos p ON v.produto_id = p.produto_id
        JOIN vendedores vd ON v.vendedor_id = vd.vendedor_id
        ORDER BY valor_total_venda DESC
        FETCH FIRST 10 ROWS ONLY
    """)

    results = cursor.fetchall()

    # Convertendo os resultados em um DataFrame
    df = pd.DataFrame(results, columns=['ID da Venda', 'Cliente', 'Produto', 'Preço', 'Quantidade', 'Valor Total da Venda', 'Vendedor', 'Data da Venda'])

    # Exibindo o DataFrame
    print(df)
except cx_Oracle.DatabaseError as e:
    print("Erro ao buscar as maiores vendas:", e)

   ID da Venda              Cliente               Produto   Preço  Quantidade  \
0           22  Cliente Genérico 14  Produto Eletrônico 1  954.15           5   
1          392  Cliente Genérico 29  Produto Eletrônico 1  954.15           5   
2          388  Cliente Genérico 27  Produto Eletrônico 1  954.15           5   
3          373  Cliente Genérico 30  Produto Eletrônico 1  954.15           5   
4           65  Cliente Genérico 20  Produto Eletrônico 1  954.15           5   
5          343  Cliente Genérico 28  Produto Eletrônico 1  954.15           5   
6          232   Cliente Genérico 8  Produto Eletrônico 1  954.15           5   
7          223  Cliente Genérico 14  Produto Eletrônico 1  954.15           5   
8          125   Cliente Genérico 6  Produto Eletrônico 1  954.15           5   
9          344  Cliente Genérico 26  Produto Eletrônico 1  954.15           5   

   Valor Total da Venda     Vendedor Data da Venda  
0               4770.75   Vendedor 3    2023-07-14  
1 

In [86]:
df

,ID da Venda,Cliente,Produto,Preço,Quantidade,Valor Total da Venda,Vendedor,Data da Venda
0,22,Cliente Genérico 14,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 3,2023-07-14
1,392,Cliente Genérico 29,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 2,2023-03-04
2,388,Cliente Genérico 27,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 10,2024-01-05
3,373,Cliente Genérico 30,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 4,2023-04-14
4,65,Cliente Genérico 20,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 9,2023-04-21
5,343,Cliente Genérico 28,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 1,2024-02-06
6,232,Cliente Genérico 8,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 2,2023-12-11
7,223,Cliente Genérico 14,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 2,2023-10-23
8,125,Cliente Genérico 6,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 4,2023-12-16
9,344,Cliente Genérico 26,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 8,2023-04-01


In [87]:
# Fazendo com o ORDER BY da Data
try:
    cursor.execute("""
        SELECT v.venda_id, c.nome AS cliente_nome, p.nome AS produto_nome, p.preco, v.quantidade,
        (p.preco * v.quantidade) AS valor_total_venda, vd.nome AS vendedor_nome, v.data_venda
        FROM vendas v
        JOIN clientes c ON v.cliente_id = c.cliente_id
        JOIN produtos p ON v.produto_id = p.produto_id
        JOIN vendedores vd ON v.vendedor_id = vd.vendedor_id
        ORDER BY v.data_venda
    """)

    results = cursor.fetchall()

    # Convertendo os resultados em um DataFrame
    df = pd.DataFrame(results, columns=['ID da Venda', 'Cliente', 'Produto', 'Preço', 'Quantidade', 'Valor Total da Venda', 'Vendedor', 'Data da Venda'])

    # Exibindo o DataFrame
    print(df.to_string(index=False))
except cx_Oracle.DatabaseError as e:
    print("Erro ao buscar as maiores vendas:", e)

 ID da Venda             Cliente               Produto  Preço  Quantidade  Valor Total da Venda    Vendedor Data da Venda
         392 Cliente Genérico 29  Produto Eletrônico 1 954.15           5               4770.75  Vendedor 2    2023-03-04
         259  Cliente Genérico 9  Produto Eletrônico 3 817.07           5               4085.35  Vendedor 2    2023-03-07
         393 Cliente Genérico 23  Produto Eletrônico 8 535.68           2               1071.36  Vendedor 8    2023-03-08
         117  Cliente Genérico 6  Produto Eletrônico 5 606.13           4               2424.52 Vendedor 10    2023-03-08
         375 Cliente Genérico 25 Produto Eletrônico 10 455.52           2                911.04  Vendedor 3    2023-03-08
          54 Cliente Genérico 20  Produto Eletrônico 5 606.13           1                606.13  Vendedor 6    2023-03-08
         213 Cliente Genérico 19  Produto Eletrônico 9 184.82           1                184.82  Vendedor 7    2023-03-09
         171 Cliente Gen

In [88]:
df

,ID da Venda,Cliente,Produto,Preço,Quantidade,Valor Total da Venda,Vendedor,Data da Venda
0,392,Cliente Genérico 29,Produto Eletrônico 1,954.15,5,4770.75,Vendedor 2,2023-03-04
1,259,Cliente Genérico 9,Produto Eletrônico 3,817.07,5,4085.35,Vendedor 2,2023-03-07
2,393,Cliente Genérico 23,Produto Eletrônico 8,535.68,2,1071.36,Vendedor 8,2023-03-08
3,117,Cliente Genérico 6,Produto Eletrônico 5,606.13,4,2424.52,Vendedor 10,2023-03-08
4,375,Cliente Genérico 25,Produto Eletrônico 10,455.52,2,911.04,Vendedor 3,2023-03-08
...,...,...,...,...,...,...,...,...
394,72,Cliente Genérico 7,Produto Eletrônico 2,128.36,1,128.36,Vendedor 7,2024-02-29
395,128,Cliente Genérico 26,Produto Eletrônico 9,184.82,4,739.28,Vendedor 5,2024-02-29
396,34,Cliente Genérico 25,Produto Eletrônico 1,954.15,2,1908.30,Vendedor 4,2024-02-29
397,262,Cliente Genérico 1,Produto Eletrônico 7,823.36,2,1646.72,Vendedor 9,2024-03-01


In [90]:
data = input("Digite da data: ")
formato = "%Y-%m-%d"
data_datetime = datetime.strptime(data, formato)
produto = input("Digite o código do produto: ")

try:
    cursor.execute("""
        SELECT v.venda_id, c.nome AS cliente_nome, p.nome AS produto_nome, p.preco, v.quantidade,
        (p.preco * v.quantidade) AS valor_total_venda, vd.nome AS vendedor_nome, v.data_venda
        FROM vendas v
        JOIN clientes c ON v.cliente_id = c.cliente_id
        JOIN produtos p ON v.produto_id = p.produto_id
        JOIN vendedores vd ON v.vendedor_id = vd.vendedor_id
        WHERE v.data_venda = :data_datetime -- AND v.produto_id = :produto
        ORDER BY v.data_venda DESC
    """, [data_datetime])

    results = cursor.fetchall()

    if results:
        df = pd.DataFrame(results, columns=['ID Venda', 'Nome Cliente', 'Nome Produto', 'Preço', 'Quantidade', 'Valor Total Venda', 'Nome Vendedor', 'Data Venda'])
        print(df)
    else:
        print("Nenhuma compra encontrada para a data e/ou produto informado.")
except cx_Oracle.DatabaseError as e:
    print("Erro ao buscar compras da data:", e)

Digite da data: 2024-01-27
Digite o código do produto: 
   ID Venda         Nome Cliente           Nome Produto   Preço  Quantidade  \
0        50  Cliente Genérico 10   Produto Eletrônico 1  954.15           1   
1       159   Cliente Genérico 5   Produto Eletrônico 5  606.13           5   
2       326  Cliente Genérico 30  Produto Eletrônico 10  455.52           3   

   Valor Total Venda Nome Vendedor Data Venda  
0             954.15    Vendedor 5 2024-01-27  
1            3030.65    Vendedor 5 2024-01-27  
2            1366.56    Vendedor 5 2024-01-27  


In [91]:
df

,ID Venda,Nome Cliente,Nome Produto,Preço,Quantidade,Valor Total Venda,Nome Vendedor,Data Venda
0,50,Cliente Genérico 10,Produto Eletrônico 1,954.15,1,954.15,Vendedor 5,2024-01-27
1,159,Cliente Genérico 5,Produto Eletrônico 5,606.13,5,3030.65,Vendedor 5,2024-01-27
2,326,Cliente Genérico 30,Produto Eletrônico 10,455.52,3,1366.56,Vendedor 5,2024-01-27


In [92]:
df.columns

Index(['ID Venda', 'Nome Cliente', 'Nome Produto', 'Preço', 'Quantidade',
       'Valor Total Venda', 'Nome Vendedor', 'Data Venda'],
      dtype='object')

In [96]:
df.groupby('Data Venda')['Valor Total Venda'].describe()

,count,mean,std,min,25%,50%,75%,max
Data Venda,,,,,,,,
2024-01-27,3.0,1783.786667,1099.327809,954.15,1160.355,1366.56,2198.605,3030.65


In [82]:
# Inserindo dados em 'vendas' com valores aleatórios
for i in range(202, 401):
    data_venda = datetime.now() - timedelta(days=random.randint(1, 365))
    cursor.execute(f"""
        INSERT INTO vendas (venda_id, cliente_id, produto_id, vendedor_id, quantidade, data_venda)
        VALUES ({i}, {random.randint(1, 30)}, {random.randint(1, 10)}, {random.randint(1, 10)}, {random.randint(1, 5)}, TO_DATE('{data_venda.strftime('%Y-%m-%d')}', 'YYYY-MM-DD'))
    """)
connection.commit()